# Magenta

Generating music via RNN's

* info: [https://magenta.tensorflow.org/](https://magenta.tensorflow.org/)

* github: [https://github.com/tensorflow/magenta](https://github.com/tensorflow/magenta)

* models: [https://github.com/tensorflow/magenta/tree/master/magenta/models](https://github.com/tensorflow/magenta/tree/master/magenta/models)


## Setup

* Activate your conda environment
* Install the Magenta pip package: `pip install magenta`

Linux: Note that in order to install the `rtmidi` package that we depend on, you may need to install headers for some sound libraries. On Linux, this command should install the necessary packages:
* `sudo apt-get install build-essential libasound2-dev libjack-dev`


## Drums RNN

This will generate a drum track. If you'd like, you can also supply priming drums using a string representation of a Python list. The values in the list should be tuples of integer MIDI pitches representing the drums that are played simultaneously at each step. For example --primer_drums="[(36, 42), (), (42,)]" would prime the model with one step of bass drum and hi-hat, then one step of rest, then one step of just hi-hat. Instead of using --primer_drums, we can use --primer_midi to prime our model with drums stored in a MIDI file.

In [1]:
%system drums_rnn_generate \
--config='drum_kit' \
--bundle_file=./magentaBundles/drum_kit_rnn.mag \
--output_dir=./midiFiles/drumLoops/ \
--num_outputs=10 \
--num_steps=128 \
--primer_drums="[(36, 42), (), (42,)]"

['/Users/aaronsherwood/anaconda2/envs/magenta/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.',
 '  from ._conv import register_converters as _register_converters',
 '2018-04-11 11:09:33.355499: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA',
 'WARNING:tensorflow:The saved meta_graph is possibly from an older release:',
 "'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.",
 'INFO:tensorflow:Restoring parameters from /var/folders/2z/khrnj_dn3zv_t52wp04myk200000gn/T/tmp_2kTVj/model.ckpt',
 '2018-04-11 11:09:33.685841: W tensorflow/core/framework/op_def_util.cc:343] Op TensorArray is deprecated. It will cease to work in GraphDef version 16. Use TensorArrayV3

## Melody RNN

This model applies language modeling to melody generation using an LSTM.

## Configurations

### Basic

This configuration acts as a baseline for melody generation with an LSTM model. It uses basic one-hot encoding to represent extracted melodies as input to the LSTM.

### Lookback

Lookback RNN introduces custom inputs and labels. The custom inputs allow the model to more easily recognize patterns that occur across 1 and 2 bars. They also help the model recognize patterns related to an events position within the measure. The custom labels reduce the amount of information that the RNN’s cell state has to remember by allowing the model to more easily repeat events from 1 and 2 bars ago. This results in melodies that wander less and have a more musical structure. For more information about the custom inputs and labels, and to hear some generated sample melodies, check out the [blog post](https://magenta.tensorflow.org/2016/07/15/lookback-rnn-attention-rnn/). You can also read through the `events_to_input` and `events_to_label` methods in `magenta/music/encoder_decoder.py` and `magenta/music/melody_encoder_decoder.py` to see how the custom inputs and labels are actually being encoded.

### Attention

In this configuration we introduce the use of attention. Attention allows the model to more easily access past information without having to store that information in the RNN cell's state. This allows the model to more easily learn longer term dependencies, and results in melodies that have longer arching themes. For an overview of how the attention mechanism works and to hear some generated sample melodies, check out the [blog post](https://magenta.tensorflow.org/2016/07/15/lookback-rnn-attention-rnn/). You can also read through the `AttentionCellWrapper` code in Tensorflow to see what's really going on under the hood.

## Usage

The script in the cell below will generate a melody starting with a middle C. If you'd like, you can also supply a priming melody using a string representation of a Python list. The values in the list should be ints that follow the melodies_lib.Melody format (-2 = no event, -1 = note-off event, values 0 through 127 = note-on event for that MIDI pitch). For example `--primer_melody="[60, -2, 60, -2, 67, -2, 67, -2]"` would prime the model with the first four notes of Twinkle Twinkle Little Star. Instead of using --primer_melody, we can use --primer_midi to prime our model with a melody stored in a MIDI file. For example, `--primer_midi=<absolute path to magenta/models/melody_rnn/primer.mid>` will prime the model with the melody in that MIDI file.

**Bundle and config should be one of 'basic_rnn', 'lookback_rnn', or 'attention_rnn', with the bundle file ending in .mag**.

In [16]:
%system melody_rnn_generate \
--config='attention_rnn' \
--bundle_file=./magentaBundles/attention_rnn.mag \
--output_dir=./midiFiles/melody_rnn/ \
--num_outputs=10 \
--num_steps=128 \
--primer_melody="[60]"

## Polyphony RNN

This model applies language modeling to polyphonic music generation using an LSTM. Unlike melodies, this model needs to be capable of modeling multiple simultaneous notes. The pre-rained model has been trained on Bach Chorales.

The following will generate a polyphonic sequence using a C Major chord as a primer.

In [15]:
%system polyphony_rnn_generate \
--bundle_file=./magentaBundles/polyphony_rnn.mag \
--output_dir=./midiFiles/polyphony_rnn/ \
--num_outputs=10 \
--num_steps=128 \
--primer_pitches="[67,64,60]" \
--condition_on_primer=true \
--inject_primer_during_generation=false

There are several command line options for controlling the generation process:

* **primer_pitches**: A string representation of a Python list of pitches that will be used as a starting chord with a quarter note duration. For example: ```"[60, 64, 67]"```.
* **primer_melody**: A string representation of a Python list of `magenta.music.Melody` event values (-2 = no event, -1 = note-off event, values 0 through 127 = note-on event for that MIDI pitch). For example: `"[60, -2, 60, -2, 67, -2, 67, -2]"`.
* **primer_midi**: The path to a MIDI file containing a polyphonic track that will be used as a priming track.
* **condition_on_primer**: If set, the RNN will receive the primer as its input before it begins generating a new sequence. You most likely want this to be true if you're using **primer_pitches** to start the sequence with a chord to establish a certain key. If you're using **primer_melody** because you want to inject a melody into the output using **inject_primer_during_generation**, you likely want this to be false, otherwise the model will see a monophonic melody before being asked to produce a polyphonic sequence. However, it may be interesting to experiment with this being on or off for each of those cases.
* **inject_primer_during_generation**: If set, the primer will be injected as a part of the generated sequence. This option is useful if you want the model to harmonize an existing melody. This option will most likely be used with **primer_melody** and `--condition_on_primer=false`.

Here's another example that will harmonize the first few notes of *Twinkle, Twinkle, Little Star*:

In [18]:
%system polyphony_rnn_generate \
--bundle_file=./magentaBundles/polyphony_rnn.mag \
--output_dir=./midiFiles/polyphony_rnn/ \
--num_outputs=10 \
--num_steps=64 \
--primer_melody="[60, -2, -2, -2, 60, -2, -2, -2, "\
"67, -2, -2, -2, 67, -2, -2, -2, 69, -2, -2, -2, "\
"69, -2, -2, -2, 67, -2, -2, -2, -2, -2, -2, -2]" \
--condition_on_primer=false \
--inject_primer_during_generation=true

## Improv RNN

This model generates melodies a la [Melody RNN](/magenta/models/melody_rnn/README.md), but conditions the melodies on an underlying chord progression. At each step of generation, the model is also given the current chord as input (encoded as a vector). Instead of training on MIDI files, the model is trained on lead sheets in MusicXML format.

## Configurations

### Basic Improv

This configuration is similar to the basic Melody RNN, but also provides the current chord encoded as a one-hot vector of 48 triads (major/minor/augmented/diminished for all 12 root pitch classes).

### Attention Improv

This configuration is similar to the attention Melody RNN, but also provides the current chord encoded as a one-hot vector of the 48 triads.

### Chord Pitches Improv

This configuration is similar to Basic Improv, but instead of using a one-hot encoding for chord triads, encodes a chord as the concatenation of the following length-12 vectors:

* a one-hot encoding of the chord root pitch class, e.g. `[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]` for a D major (or minor, etc.) chord
* a binary vector indicating presence or absence of each pitch class, e.g. `[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0]` for a C7#9 chord
* a one-hot encoding of the chord bass pitch class, which is usually the same as the chord root pitch class except in the case of "slash chords" like C/E

## Usage

The script in the cell below will generate a melody starting with a middle C over the chord progression C G Am F, where each chord lasts one bar and the progression is repeated twice. If you'd like, you can supply a longer priming melody using a string representation of a Python list. The values in the list should be ints that follow the melodies_lib.Melody format (-2 = no event, -1 = note-off event, values 0 through 127 = note-on event for that MIDI pitch). For example `--primer_melody="[60, -2, 60, -2, 67, -2, 67, -2]"` would prime the model with the first four notes of *Twinkle Twinkle Little Star*. Instead of using `--primer_melody`, we can use `--primer_midi` to prime our model with a melody stored in a MIDI file. For example, `--primer_midi=<absolute path to magenta/models/melody_rnn/primer.mid>` will prime the model with the melody in that MIDI file.

You can modify the backing chords as you like; Magenta understands most basic chord types e.g. "A13", "Cdim", "F#m7b5". The `--steps_per_chord` option can be used to control the chord duration.

In [26]:
%system improv_rnn_generate \
--config='attention_improv' \
--bundle_file=./magentaBundles/chord_pitches_improv.mag \
--output_dir=./midiFiles/improv_rnn/ \
--num_outputs=10 \
--primer_melody="[60]" \
--backing_chords="C G Am F C G Am F" \
--render_chords

## Performance RNN

Performance RNN models polyphonic performances with dynamics and expressive timing. It uses an event sequence encoding like [Polyphony RNN](/models/polyphony_rnn/README.md) but with the following event types:

* NOTE_ON(*pitch*): start a note at *pitch*
* NOTE_OFF(*pitch*): stop a note at *pitch*
* TIME_SHIFT(*amount*): advance time by *amount*
* VELOCITY(*value*): change current velocity to *value*

This model creates music in a language similar to MIDI itself, with **note-on** and **note-off** events instead of explicit durations. In order to support expressive timing, the model controls the clock with **time-shift** events that move forward at increments of 10 ms, up to 1 second. All note-on and note-off events happen at the current time as determined by all previous time shifts in the event sequence. The model also supports **velocity** events that set the current velocity, used by subsequent note-on events.  Velocity can optionally be quantized into fewer than the 127 valid MIDI velocities.

Because of this representation, the model is capable of generating performances with more natural timing and dynamics compared to our other models that a) use a quantized metrical grid with fixed tempo and b) don't handle velocity.

At generation time, a few undesired behaviors can occur: note-off events with no previous note-on (these are ignored), and note-on events with no subsequent note-off (these are ended after 5 seconds).

The latter three models are *conditional* models that can generate performances conditioned on desired note density, desired pitch class distribution, or both, respectively.

### Generate a performance

Bundle and config should be one of 'performance', 'performance_with_dynamics', 'density_conditioned_performance_with_dynamics', 'pitch_conditioned_performance_with_dynamics', or 'multiconditioned_performance_with_dynamics', with the bundle file ending in .mag (***performance.mag not included in this repo***).

The script in the cell below will generate a performance starting with an ascending C major scale.

There are several command-line options for controlling the generation process:

* **primer_pitches**: A string representation of a Python list of pitches that will be used as a starting chord with a short duration. For example: ```"[60, 64, 67]"```.
* **primer_melody**: A string representation of a Python list of `magenta.music.Melody` event values (-2 = no event, -1 = note-off event, values 0 through 127 = note-on event for that MIDI pitch). For example: `"[60, -2, 60, -2, 67, -2, 67, -2]"`.
* **primer_midi**: The path to a MIDI file containing a polyphonic track that will be used as a priming track.

If you're using one of the conditional models, there are additional command-line options you can use:

* **notes_per_second**: The desired number of notes per second in the output performance. Note that increasing this value will cause generation to take longer, as the number of RNN steps is roughly proportional to the number of notes generated.
* **pitch_class_histogram**: A string representation of a Python list of 12 values representing the relative frequency of notes of each pitch class, starting with C. For example: `"[2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]"` will tend to stick to a C-major scale, with twice as much C as any of the other notes of the scale.

These control variables are not strictly enforced, but can be used to guide the model's output. Currently these can only be set globally, affecting the entire performance.

In [30]:
%system performance_rnn_generate \
--config='performance_with_dynamics' \
--bundle_file=./magentaBundles/performance_with_dynamics.mag \
--output_dir=./midiFiles/performance_rnn/\
--num_outputs=1 \
--num_steps=3000 \
--primer_melody="[60,62,64,65,67,69,71,72]"

In [21]:
# to clear outputs from any cell past this at the bottom of the cell and run:
print("doesn't look like anything to me")
clear_output()

[Extract midi from Garageband](http://scotttroyer.com/2014/05/export-midi-from-garageband/)